Suppose that we are given two coins A and B with unkown bias $\theta_A$ and $\theta_B$, respectively. Our goal is to estimate the bias vector $\boldsymbol{\theta}= (\theta_A, \theta_B)$ from the outcomes of the following experiment: 

<blockquote> 
First choose one coin at random. Then toss the selected coin 10 times independently and record the number of heads observed. Repeat this procedure 5 times.
</blockquote>

Formally, let $z_i\in\{A,B\}$ be the coin selected in experiment $i$ and $x_i\in\{0,1,\cdots 10\}$ be the number heads recorded by tossing $z_i$ 10 times. Since we conduct $n=5$ such experiments, we can summarize the outcomes of these 50 tosses by two vectors: $\boldsymbol{x}=(x_1,x_2\cdots, x_5)$ and $\boldsymbol{z}=(z_1,z_2,\cdots, z_5)$.

### What if we know everything?

 * Consider first the case where we have complete knowledge of the experiment, namely, both $\boldsymbol{x}$ and $\boldsymbol{z}$ are known. How would you intuitively estimate the biases of the two coins  $\boldsymbol{\theta}= (\theta_A, \theta_B)$ ?
 
 * What's the likelihood of observing the complete outcomes of these experiments? In other words, what is $P(\boldsymbol{x},\boldsymbol{z}| n,\boldsymbol{\theta} )$? You may assume this is a Bernoulli trial. Namely, every time coin A(B) is tossed, we have, with probability $\theta_A$($\theta_B$), that the outcome is heads.
 
 * What's the Maximum Likelihood Estimator (MLE)? Is this consistent with your intuition? 


## Comparing MLE and EM

To test your answer, let's do some numerics! We will compare the MLE estimates of biases with an Expectation Maximization procedure where we do not know ${\bf z}$. The following code computes our best guess for the biases using MLE -- assuming we know the identity of the coin used -- and compares it estimates arrived at using an EM procedure where we have no knowledge about which coin was being tossed (though we know the same coin was tossed 10 times).

In [1]:
import numpy as np
from scipy.special import comb

Computing Maximum Likelihood Estimation

In [2]:
n_coin_tosses = 10
n_expts = 5
thetaA_heads = 0.8
thetaB_heads = 0.4

coin_choice = np.zeros(n_expts)
head_counts = np.zeros(n_expts)
    
MLE_A = 0.0
MLE_B = 0.0

for expt in range(n_expts):
    if np.random.randint(2) == 0: ## coin A is selected
        coin_choice[expt] = 0
        head_counts[expt] = np.random.binomial(n_coin_tosses, thetaA_heads, 1)
        MLE_A += head_counts[expt]
    else: ## coin B is selected
        coin_choice[expt] = 1
        head_counts[expt] = np.random.binomial(n_coin_tosses, thetaB_heads, 1)
        MLE_B += head_counts[expt]
    
MLE_A = MLE_A / ((n_expts - np.count_nonzero(coin_choice))*n_coin_tosses)
MLE_B = MLE_B / (np.count_nonzero(coin_choice)*n_coin_tosses)

Expectation Maximization

In [3]:
# This function will come in use below to compute the likelihood for the random binomial distribution

def compute_likelihood(n, k, pheads): 

    likelihood = comb(n, k, exact=True)*(pheads**k)*(1.0-pheads)**(n-k)

    return likelihood

In [4]:
# Initialize the coin biases pA_heads = pA(heads), pB_heads = pB(heads)
pA_heads = []
pB_heads = []

# random initialization
pA_heads.append(0.60)
pB_heads.append(0.50)

# EM begins now
epsilon = 0.001 # error threshold
j = 0  #improvement/epoch counter
improvement = float('inf')

while improvement > epsilon:
    expectation_A = np.zeros((n_expts, 2), dtype=float)
    expectation_B = np.zeros((n_expts, 2), dtype=float)
    
    for expt in range(n_expts):
        eH = head_counts[expt]
        eT = n_coin_tosses - head_counts[expt]
        
        # Expectation Step
        lA = compute_likelihood(n_coin_tosses, eH, pA_heads[j])
        lB = compute_likelihood(n_coin_tosses, eH, pB_heads[j])
        
        weightA = lA / (lA + lB)
        weightB = lB / (lA + lB)
        
        expectation_A[expt] = np.multiply(weightA, [eH, eT])
        expectation_B[expt] = np.multiply(weightB, [eH, eT])
    
    # Maximization Step
    thetaA = np.sum(expectation_A, axis=0)[0] / np.sum(expectation_A)
    thetaB = np.sum(expectation_B, axis=0)[0] / np.sum(expectation_B)
    print("At iteration %d: thetaA = %2f,  thetaB = %2f" % (j, thetaA, thetaB))
    
    pA_heads.append(thetaA)
    pB_heads.append(thetaB)
    
    improvement = max(abs(np.array([pA_heads[j+1], pB_heads[j+1]]) - np.array([pA_heads[j], pB_heads[j]])))
    
    j += 1
    
# END of E-M, print the outcome
print("")
print("E-M converges at iteration %d" %j)
print("RESULT of E-M:")
print("theta_A = %2f,  theta_B = %2f" % (thetaA, thetaB))
print("MLE with complete data: theta_A = %2f,  theta_B = %2f" % (MLE_A, MLE_B))

At iteration 0: thetaA = 0.836770,  thetaB = 0.651211
At iteration 1: thetaA = 0.901389,  thetaB = 0.604021
At iteration 2: thetaA = 0.919082,  thetaB = 0.573289
At iteration 3: thetaA = 0.922492,  thetaB = 0.564862
At iteration 4: thetaA = 0.923004,  thetaB = 0.562855
At iteration 5: thetaA = 0.923036,  thetaB = 0.562315

E-M converges at iteration 6
RESULT of E-M:
theta_A = 0.923036,  theta_B = 0.562315
MLE with complete data: theta_A = 0.933333,  theta_B = 0.550000
